# Train remotely in Azure Machine Learning

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
SUBSCRIPTION_ID = os.getenv("SUBSCRIPTION_ID") 
RESOURCE_GROUP = os.getenv("RESOURCE_GROUP")
AML_WORKSPACE_NAME = os.getenv("AML_WORKSPACE_NAME")

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION_ID,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=AML_WORKSPACE_NAME,
)


Class WorkspaceHubOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [3]:
#create  or get compute
from azure.ai.ml.entities import AmlCompute

cpu_compute_name = "iris-cpu-cluster"

try:
    ml_client.compute.get(cpu_compute_name)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_name, size="STANDARD_D2_V2", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()

In [4]:
# create or get dataset
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# local filesystem
data_asset_name = "Iris"
my_path = "./data/iris.csv"
# set the version number of the data asset
v1 = "1"

my_data = Data(
    name=data_asset_name,
    version=v1,
    description="Iris data set",
    path=my_path,
    type=AssetTypes.URI_FILE,
)

## create data asset if it doesn't already exist:
try:
    data_asset = ml_client.data.get(name=data_asset_name, version=v1)
    print(
        f"Data asset already exists. Name: {my_data.name}, version: {my_data.version}"
    )
except:
    data_asset = ml_client.data.create_or_update(my_data)
    print(f"Data asset created. Name: {my_data.name}, version: {my_data.version}")

Uploading iris.csv (< 1 MB): 100%|##########| 4.47k/4.47k [00:00<00:00, 45.9kB/s]




Data asset created. Name: Iris, version: 1


In [5]:
from azure.ai.ml import command, Input

# define the command
command_job = command(
    code="./src",
    command="python main.py --iris-csv ${{inputs.iris_csv}} --learning-rate ${{inputs.learning_rate}} --boosting ${{inputs.boosting}}",
    environment="AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu@latest",
    inputs={
        "iris_csv": Input(
            type="uri_file",
            path=data_asset.path,
        ),
        "learning_rate": 0.9,
        "boosting": "gbdt",
    },
    compute=cpu_compute_name,
)

In [6]:
# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


'https://ml.azure.com/runs/dynamic_oregano_vhktvrfy83?wsid=/subscriptions/ec967cb5-f2b0-43c2-9ba2-4a2eb94bbacc/resourcegroups/MLOps-demo-azureml/workspaces/modelops-demo-azureml&tid=16b3c013-d300-468d-ac64-7eda0820b6d3'

## Register the trained model

In [7]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

run_model = Model(
    path="azureml://jobs/{}/outputs/artifacts/paths/model/".format(returned_job.name),
    name="run-model-example",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL
)

ml_client.models.create_or_update(run_model)

Model({'job_name': 'dynamic_oregano_vhktvrfy83', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'run-model-example', 'description': 'Model created from run.', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/ec967cb5-f2b0-43c2-9ba2-4a2eb94bbacc/resourceGroups/MLOps-demo-azureml/providers/Microsoft.MachineLearningServices/workspaces/modelops-demo-azureml/models/run-model-example/versions/2', 'Resource__source_path': None, 'base_path': 'c:\\Users\\dschlesinger\\code\\ongoing\\Azure ML\\code\\train remotely', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000029873F5E230>, 'serialize': <msrest.serialization.Serializer object at 0x0000029873F5E4D0>, 'version': '2', 'latest_version': None, 'path': 'azureml://subscriptions/ec967cb5-f2b0-43c2-9ba2-4a2eb94bbacc/resourceGroups/MLOps-demo-azureml/workspaces/modelops-demo-azureml/datastores/workspaceartifactstore/